In [1]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import pickle
import warnings
warnings.filterwarnings('ignore')

In [2]:
Client_ID = '51cc939834024272ab5f096dd12f7940'
Client_Secret = '7a01a6a8bdbb4e9dbe3a30f4b9a7f891' 


sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= Client_ID,
                                                           client_secret= Client_Secret))

In [3]:
spoti_data = pd.read_csv('./one_data.csv')
# almost_data = spoti_data.set_index('id')
data = spoti_data.reindex(columns= ["danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness",
                                     "instrumentalness", "liveness", "valence", "tempo", "duration_ms","title","artist","id"])

In [4]:
data

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,title,artist,id
0,0.708,0.1950,10,-12.428,1,0.0506,0.9950,0.563000,0.1510,0.7790,118.469,158648,Singende Bataillone 1. Teil,['Carl Woitschach'],6KbQ3uYMLKb5jDxLF7wYDD
1,0.379,0.0135,8,-28.454,1,0.0462,0.9940,0.901000,0.0763,0.0767,83.972,282133,"Fantasiestücke, Op. 111: Più tosto lento","['Robert Schumann', 'Vladimir Horowitz']",6KuQTIu1KoTTkLXKrwlLPV
2,0.749,0.2200,5,-19.924,0,0.9290,0.6040,0.000000,0.1190,0.8800,107.177,104300,Chapter 1.18 - Zamek kaniowski,['Seweryn Goszczyński'],6L63VW0PibdM1HDSBoqnoM
3,0.781,0.1300,1,-14.734,0,0.0926,0.9950,0.887000,0.1110,0.7200,108.003,180760,Bebamos Juntos - Instrumental (Remasterizado),['Francisco Canaro'],6M94FkXd15sOAOQYRnWPN8
4,0.210,0.2040,11,-16.829,1,0.0424,0.9900,0.908000,0.0980,0.0693,62.149,687733,"Polonaise-Fantaisie in A-Flat Major, Op. 61","['Frédéric Chopin', 'Vladimir Horowitz']",6N6tiFZ9vLTSOIxkj8qKrd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
169875,0.875,0.4430,1,-7.461,1,0.1430,0.1730,0.000032,0.0891,0.3060,100.012,163800,Skechers (feat. Tyga) - Remix,"['DripReport', 'Tyga']",4KppkflX7I3vJQk7urOJaS
169876,0.719,0.3850,8,-10.907,1,0.0403,0.0167,0.031300,0.1110,0.2700,128.000,167468,Sweeter (feat. Terrace Martin),"['Leon Bridges', 'Terrace Martin']",1ehhGlTvjtHo2e4xJFB0SZ
169877,0.514,0.5390,7,-9.332,1,0.1050,0.5380,0.002330,0.1080,0.1530,123.700,180700,How Would I Know,"['Kygo', 'Oh Wonder']",52eycxprLhK3lPcRLbQiVk
169878,0.646,0.7610,1,-2.557,1,0.0385,0.0714,0.000000,0.2220,0.4720,129.916,167308,I Found You,"['Cash Cash', 'Andy Grammer']",3wYOGJYD31sLRmBgCvWxa4


In [20]:
def if_not_hot():
########################## Ask the user for input
    print("Tell me a song that you like\n")
    
    state = True
    while state == True: ########## While loop to loop over it till the condition state = False. So it asks for input till the user enters a valid song in Spotify
        song = input()
        
    ########################## Call Spotify API to find input song   
        results = sp.search(q=song, limit=50)
        json_results = json.dumps(results, ensure_ascii=False)

    ########################### TEMPORAL: Create a list with the different artists for the song input. 
        artiste_list = []

        for i in range(len(results["tracks"]["items"])):
            artistas = results["tracks"]["items"][i]["artists"][0]["name"]
        
            if artistas not in artiste_list:
                artiste_list.append(artistas)

    ############################ While loop that will loop till the user inputs a song that's in the Spotify database.

        if len(artiste_list) != (0):            
            print("\nWhat's the artist?\n")      
            art = input() #### Ask the user to choose the artist for the entered song.
            print()
            words = art.split(' ') # Capitalization of user_input to fit with hot_list
            capi = [word.capitalize() for word in words]
            correction = ' '.join(capi)
            
            

            artist = True
            while artist == True: ### While loop to loop till the user enters a correct input for the artist of the song
                if correction in artiste_list:
                    i = artiste_list.index(correction)
                    features = sp.audio_features(results["tracks"]["items"][i]['uri']) # If the artist name is correct/in the database, it will return the features for the song.
                    artist = False
                    state = False
                   

                else:
                    print("Sorry, we don't have {} by {} in our dataset \n".format(song, art)) # Otherwise it will ask for another input, and will give an artist list for that song.
                    print("Why don't you try with a different artist from this list? \n")
                    print(artiste_list)
                    art = input()
                   
            state = False
        else:

            print("Sorry, the song you input is not in our database or it's misspelled. Please try again.\n")

############################ Remove unnecesary features
    to_remove =("type", "id", "uri", "track_href", "analysis_url","time_signature")
    unwanted_features = [features[0].pop(x) for x in to_remove]

############################ Import pickle 'scaler' + transform
    infile = open("scaling",'rb')
    scaler = pickle.load(infile)
    infile.close()
    
    transformed_features = scaler.transform(pd.DataFrame(features))
############################ Import picke model + prediction
    infile = open("model",'rb')
    model = pickle.load(infile)
    infile.close()
    
    recommendation = model.predict(transformed_features)
    return int(recommendation)

In [6]:
if_not_hot()

Tell me a song that you like



 little wing



What's the artist?



 jimi hendrix


3

In [7]:
### Create dataframe for recommendation function

In [23]:
infile = open("scaling",'rb')
scaler = pickle.load(infile)
infile.close()
infile = open("model",'rb')
model = pickle.load(infile)
infile.close()


In [24]:
to_cluster = scaler.transform(data.select_dtypes(include='number'))
clusters = model.predict(to_cluster)

In [26]:
clustered_data = data.select_dtypes(include='object')

In [27]:
clustered_data ['cluster'] = clusters

In [ ]:
df = hot_songs_spotify()

df['song'] = df['artist'].apply(lambda x: x.split('(')[0].lower())

df['song'] = [x.rstrip() for x in df['song']]

In [29]:
clustered_data['artist'] = [x.replace("[", "") for x in clustered_data['artist']]
clustered_data['artist'] = [x.replace("]", "") for x in clustered_data['artist']]
clustered_data['artist'] = [x.replace("'", "") for x in clustered_data['artist']]



In [30]:
clustered_data.to_csv('to_work.csv', index=False)

In [ ]:
clustered_data

In [ ]:
def song_classify(song_cluster, clustered_data):
    to_recommend = clustered_data[clustered_data['cluster']==song_cluster]    
    print("Cool! See if you like:\n")
    
    nada = True
    while nada == True:
        recommendation = to_recommend.sample()
        print(recommendation['title'].to_string(index=False) +'  by '+ recommendation['artist'].to_string(index=False)+"\n")
        print('How do you find this recommendation? Good or bad?\n')
        response = input().lower()
        print()
        
    
        if response == 'good':
            print("Great! See you later!\n")
            nada = False
        elif response == 'bad':
            print("Sorry to hear that, see if you like this one then\n")
        else:
            print("Sorry I cannot understand. Try again later\n")
            break
            
            
    
    

In [ ]:
song_classify(if_not_hot(), clustered_data)

In [ ]:
clustered_data[clustered_data['cluster']==5]


In [10]:
from untitled import hot_songs_spotify

In [14]:
df = hot_songs_spotify()

df['song'] = df['artist'].apply(lambda x: x.split('(')[0].lower())

df['song'] = [x.rstrip() for x in df['song']]

In [15]:
df.to_csv('hot_data.csv')